In [ ]:
# Part B: Person Following with Obstacle Avoidance
# Combines Tutorial 3 Part B (depth-based collision avoidance) with Tutorial 5 (YOLO human detection)
# Uses depth for reliable obstacle detection and YOLO only for tracking the target human

"""
STEP 1: Display Widgets Setup
Based on Tutorial 2 Part A
"""
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

display_color = widgets.Image(format='jpeg', width='45%')
display_depth = widgets.Image(format='jpeg', width='45%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])


"""
STEP 2: Load YOLO Model
Based on Tutorial 5
"""
from ultralytics import YOLO

model = YOLO("yolo11l_half.engine")


"""
STEP 3: Person Re-Identification System
HSV histogram matching to track same person across frames (handles occlusion)
"""
import numpy as np

class PersonTracker:
    """
    Tracks a specific person using HSV histogram matching.
    Handles occlusions by re-identifying based on appearance.
    """
    
    def __init__(self, similarity_threshold=0.6, h_bins=16, s_bins=16):
        self.target_histogram = None
        self.target_locked = False
        self.similarity_threshold = similarity_threshold
        self.h_bins = h_bins
        self.s_bins = s_bins
        self.frames_lost = 0
        self.max_lost_frames = 30  # ~1 second at 30fps
    
    def compute_histogram(self, roi_bgr):
        """Compute normalized HSV histogram for region of interest."""
        if roi_bgr.size == 0:
            return None
        
        hsv = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv], [0, 1], None,
                           [self.h_bins, self.s_bins],
                           [0, 180, 0, 256])
        cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
        return hist
    
    def lock_target(self, frame, bbox):
        """Lock onto specific person by storing their histogram."""
        x1, y1, x2, y2 = map(int, bbox)
        roi = frame[y1:y2, x1:x2]
        
        self.target_histogram = self.compute_histogram(roi)
        if self.target_histogram is not None:
            self.target_locked = True
            self.frames_lost = 0
            return True
        return False
    
    def find_target(self, frame, detections):
        """
        Find target person among all detections using histogram matching.
        Returns (found, bbox, distance) tuple.
        """
        if not self.target_locked:
            return False, None, float('inf')
        
        best_match_score = -1
        best_bbox = None
        best_distance = float('inf')
        
        for detection in detections:
            bbox, distance = detection
            x1, y1, x2, y2 = map(int, bbox)
            roi = frame[y1:y2, x1:x2]
            
            hist = self.compute_histogram(roi)
            if hist is None:
                continue
            
            # Calculate histogram similarity
            similarity = cv2.compareHist(self.target_histogram, hist, 
                                        cv2.HISTCMP_CORREL)
            
            if similarity > best_match_score:
                best_match_score = similarity
                best_bbox = bbox
                best_distance = distance
        
        if best_match_score > self.similarity_threshold:
            self.frames_lost = 0
            return True, best_bbox, best_distance
        else:
            self.frames_lost += 1
            if self.frames_lost > self.max_lost_frames:
                self.target_locked = False
            return False, None, float('inf')
    
    def reset(self):
        """Reset tracker to lock onto new person."""
        self.target_histogram = None
        self.target_locked = False
        self.frames_lost = 0


"""
STEP 4: Human Detection with YOLO
Based on Tutorial 5 - detect all humans and measure distances
"""

def detect_all_humans(frame, depth_image, conf_threshold=0.5):
    """
    Detect all humans in frame using YOLO and calculate distances.
    Returns list of (bbox, distance) tuples.
    """
    results = model(frame, verbose=False)
    detections = []
    
    for result in results:
        for i in range(len(result.boxes.cls)):
            if result.boxes.cls[i] == 0:  # Human class only
                if result.boxes.conf[i] > conf_threshold:
                    bbox = result.boxes.xyxy[i].cpu().numpy()
                    
                    # Calculate center of bounding box
                    center_x = int((bbox[0] + bbox[2]) / 2)
                    center_y = int((bbox[1] + bbox[3]) / 2)
                    
                    # Get distance from depth image
                    if (0 <= center_y < depth_image.shape[0] and 
                        0 <= center_x < depth_image.shape[1]):
                        distance = depth_image[center_y, center_x]
                        
                        if not np.isnan(distance) and distance > 0:
                            detections.append((bbox, distance))
    
    return detections


"""
STEP 5: Depth-Based Obstacle Detection
Based on Tutorial 3 Part B - uses depth image for reliable obstacle detection
This detects ALL obstacles (people, furniture, walls) in central region
"""

def detect_obstacle_depth(depth_image):
    """
    Detect obstacles in robot's path using depth image.
    Based on Tutorial 3 Part B collision avoidance method.
    
    Returns (obstacle_detected, min_distance, obstacle_region)
    - obstacle_region: 'left', 'right', or 'center'
    """
    # Copy and clean depth image
    depth_test = depth_image.copy()
    depth_test = np.nan_to_num(depth_test, nan=0.0).astype(np.float32)
    
    # Define central area (robot's path) - Tutorial 3 Part B approach
    # Adjust these values based on your robot's configuration
    depth_test[:94, :] = 0      # Remove top area
    depth_test[282:, :] = 0     # Remove bottom area
    depth_test[:, :168] = 0     # Remove left side
    depth_test[:, 504:] = 0     # Remove right side
    
    # Filter depth values (remove noise and distant objects)
    depth_test[depth_test < 100] = 0    # Too close readings (noise)
    depth_test[depth_test > 1000] = 0   # Too far (not immediate threat)
    
    # Check if any obstacle in path
    if depth_test.max() == 0:
        return False, float('inf'), 'none'
    
    # Find minimum distance (closest obstacle)
    min_distance = depth_test[depth_test != 0].min()
    
    # Determine which region obstacle is in (left/center/right)
    # Split central area into three regions
    height, width = depth_test.shape
    left_region = depth_test[:, 168:280]
    center_region = depth_test[:, 280:392]
    right_region = depth_test[:, 392:504]
    
    # Find which region has the closest obstacle
    left_min = left_region[left_region != 0].min() if left_region[left_region != 0].size > 0 else float('inf')
    center_min = center_region[center_region != 0].min() if center_region[center_region != 0].size > 0 else float('inf')
    right_min = right_region[right_region != 0].min() if right_region[right_region != 0].size > 0 else float('inf')
    
    if center_min <= left_min and center_min <= right_min:
        obstacle_region = 'center'
    elif left_min < right_min:
        obstacle_region = 'left'
    else:
        obstacle_region = 'right'
    
    return True, min_distance, obstacle_region


"""
STEP 6: Camera System Initialization
Based on Tutorial 2 Part B and Tutorial 4 Part A
"""
import traitlets
import pyzed.sl as sl
import threading
import motors
from traitlets.config.configurable import SingletonConfigurable

robot = motors.MotorsYukon(mecanum=False)

class Camera(SingletonConfigurable):
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER
        
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)
        
        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False
        
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
    
    def _capture_frames(self):
        while self.thread_runnning_flag:
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
                self.color_value_BGRA = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())
    
    def start(self):
        if not self.thread_runnning_flag:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()
    
    def stop(self):
        if self.thread_runnning_flag:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

camera = Camera()
camera.start()


"""
STEP 7: Following Behavior with Differential Drive
Smooth curved movement using individual wheel speed control
"""

def follow_target(frame, target_bbox, target_distance, depth_colormap):
    """
    Follow target person while maintaining optimal distance.
    Uses differential drive for smooth curves (love_detection.ipynb approach).
    """
    # Calculate target position relative to center
    bbox_center_x = (target_bbox[0] + target_bbox[2]) / 2
    frame_center_x = camera.width / 2
    
    # Calculate angle offset normalized to [-1, 1]
    offset_x = bbox_center_x - frame_center_x
    max_offset = camera.width / 2
    normalized_angle = offset_x / max_offset
    
    # Convert to radians for differential drive
    angle_rad = normalized_angle * (np.pi / 4)
    turning_factor = np.cos(angle_rad)
    
    # Draw target box
    cv2.rectangle(frame, 
                  (int(target_bbox[0]), int(target_bbox[1])), 
                  (int(target_bbox[2]), int(target_bbox[3])), 
                  (0, 255, 0), 3)
    
    # Distance-based behavior
    if target_distance < 600:
        robot.backward(0.2)
        cv2.putText(depth_colormap, f'TOO CLOSE {target_distance:.0f}mm', 
                   (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        
    elif 600 <= target_distance <= 900:
        robot.stop()
        cv2.putText(depth_colormap, f'OPTIMAL {target_distance:.0f}mm', 
                   (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
    elif 900 < target_distance <= 1500:
        base_speed = 0.3
        
        if angle_rad > 0.1:  # Right
            robot.frontLeft(speed=base_speed)
            robot.backLeft(speed=base_speed)
            robot.frontRight(speed=base_speed * turning_factor)
            robot.backRight(speed=base_speed * turning_factor)
            cv2.putText(depth_colormap, f'FOLLOW RIGHT {target_distance:.0f}mm', 
                       (160, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2)
            
        elif angle_rad < -0.1:  # Left
            robot.frontLeft(speed=base_speed * turning_factor)
            robot.backLeft(speed=base_speed * turning_factor)
            robot.frontRight(speed=base_speed)
            robot.backRight(speed=base_speed)
            cv2.putText(depth_colormap, f'FOLLOW LEFT {target_distance:.0f}mm', 
                       (160, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2)
            
        else:  # Centered
            robot.forward(base_speed)
            cv2.putText(depth_colormap, f'APPROACH {target_distance:.0f}mm', 
                       (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2)
            
    else:  # > 1500mm
        base_speed = 0.5
        
        if angle_rad > 0.1:
            robot.frontLeft(speed=base_speed)
            robot.backLeft(speed=base_speed)
            robot.frontRight(speed=base_speed * turning_factor)
            robot.backRight(speed=base_speed * turning_factor)
            cv2.putText(depth_colormap, f'CHASE RIGHT {target_distance:.0f}mm', 
                       (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)
            
        elif angle_rad < -0.1:
            robot.frontLeft(speed=base_speed * turning_factor)
            robot.backLeft(speed=base_speed * turning_factor)
            robot.frontRight(speed=base_speed)
            robot.backRight(speed=base_speed)
            cv2.putText(depth_colormap, f'CHASE LEFT {target_distance:.0f}mm', 
                       (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)
            
        else:
            robot.forward(base_speed)
            cv2.putText(depth_colormap, f'CHASE {target_distance:.0f}mm', 
                       (220, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)


"""
STEP 8: Obstacle Avoidance with Navigation
Based on Tutorial 3 Part B approach - navigate around obstacles
"""

def navigate_around_obstacle(obstacle_region, obstacle_distance, depth_colormap):
    """
    Navigate around obstacle based on Tutorial 3 Part B.
    Turns away from obstacle until path is clear.
    
    Args:
        obstacle_region: 'left', 'center', or 'right'
        obstacle_distance: distance to closest obstacle in mm
    """
    # Critical distance threshold (Tutorial 3 Part B uses 400mm)
    if obstacle_distance < 400:
        # Very close - stop or back up
        robot.backward(0.2)
        cv2.putText(depth_colormap, f'EMERGENCY STOP {obstacle_distance:.0f}mm!', 
                   (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    else:
        # Navigate around based on obstacle position
        if obstacle_region == 'left':
            # Obstacle on left, turn right
            robot.right(0.4)
            cv2.putText(depth_colormap, f'AVOID LEFT {obstacle_distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            
        elif obstacle_region == 'right':
            # Obstacle on right, turn left
            robot.left(0.4)
            cv2.putText(depth_colormap, f'AVOID RIGHT {obstacle_distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            
        else:  # center
            # Obstacle ahead, turn right (default direction)
            robot.right(0.4)
            cv2.putText(depth_colormap, f'AVOID CENTER {obstacle_distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)


"""
STEP 9: Main Control Loop with State Machine
Combines target following with obstacle avoidance
"""

tracker = PersonTracker(similarity_threshold=0.6)
current_state = 'idle'  # States: 'idle', 'locking', 'following', 'avoiding'

# Keyboard input (Answer_keyboard_control.ipynb pattern)
text_input = widgets.Text(
    value='',
    placeholder='Type: l=Lock target, s=Stop, r=Reset',
    description='Control:',
    disabled=False
)

def on_text_change(change):
    """Handle keyboard commands."""
    global current_state
    
    input_value = change['new']
    
    if len(input_value) > 0:
        last_char = input_value[-1].lower()
        
        if last_char == 'l':
            current_state = 'locking'
            print('\nLOCKING - Will lock onto closest person')
            
        elif last_char == 's':
            current_state = 'idle'
            tracker.reset()
            robot.stop()
            print('\nSTOPPED - Robot halted, tracker reset')
            
        elif last_char == 'r':
            tracker.reset()
            current_state = 'idle'
            robot.stop()
            print('\nRESET - Tracker reset')

text_input.observe(on_text_change, names='value')
display(text_input)


def main_callback(change):
    """
    Main control callback - state machine for following and avoidance.
    Combines YOLO human tracking with depth-based obstacle avoidance.
    """
    global current_state
    
    frame = change['new'].copy()
    depth_image = camera.depth_image
    
    # Create depth colormap with obstacle detection visualization
    depth_test = depth_image.copy()
    depth_test = np.nan_to_num(depth_test, nan=0.0).astype(np.float32)
    
    # Show detection zone (Tutorial 3 Part B approach)
    depth_test[:94, :] = 0
    depth_test[282:, :] = 0
    depth_test[:, :168] = 0
    depth_test[:, 504:] = 0
    depth_test[depth_test < 100] = 0
    depth_test[depth_test > 1000] = 0
    
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(depth_test, alpha=0.03), 
        cv2.COLORMAP_JET
    )
    
    # Detect all humans using YOLO
    all_detections = detect_all_humans(frame, depth_image)
    
    # Draw all detected humans in blue
    for bbox, distance in all_detections:
        cv2.rectangle(frame, 
                     (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), 
                     (255, 0, 0), 2)
    
    # Detect obstacles using depth (Tutorial 3 Part B method)
    obstacle_detected, obstacle_distance, obstacle_region = detect_obstacle_depth(depth_image)
    
    # State machine
    if current_state == 'idle':
        robot.stop()
        cv2.putText(frame, 'IDLE - Press L to lock target', 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(depth_colormap, 'IDLE', 
                   (270, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)
    
    elif current_state == 'locking':
        if len(all_detections) > 0:
            # Lock onto closest person
            closest_detection = min(all_detections, key=lambda x: x[1])
            closest_bbox, closest_distance = closest_detection
            
            if tracker.lock_target(frame, closest_bbox):
                current_state = 'following'
                print(f'\nTARGET LOCKED at {closest_distance:.0f}mm')
                
                cv2.rectangle(frame, 
                            (int(closest_bbox[0]), int(closest_bbox[1])), 
                            (int(closest_bbox[2]), int(closest_bbox[3])), 
                            (0, 255, 0), 3)
        else:
            cv2.putText(frame, 'LOCKING - No person detected', 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)
    
    elif current_state == 'following':
        # Find target person using YOLO + histogram matching
        target_found, target_bbox, target_distance = tracker.find_target(frame, all_detections)
        
        if target_found:
            # Check for obstacles using depth (more reliable than YOLO for obstacles)
            if obstacle_detected and obstacle_distance < 700:  # 700mm safety threshold
                # Obstacle in path - switch to avoiding
                current_state = 'avoiding'
                cv2.putText(frame, f'OBSTACLE DETECTED - AVOIDING', 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                # Path clear - follow target
                follow_target(frame, target_bbox, target_distance, depth_colormap)
                cv2.putText(frame, f'FOLLOWING - Distance: {target_distance:.0f}mm', 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            # Lost target - search by rotating
            robot.spinRight(0.3)
            cv2.putText(frame, f'SEARCHING ({tracker.frames_lost} frames)', 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)
            cv2.putText(depth_colormap, 'SEARCHING', 
                       (220, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2)
            
            if tracker.frames_lost > tracker.max_lost_frames:
                current_state = 'idle'
                robot.stop()
                print('\nTARGET LOST - Returning to idle')
    
    elif current_state == 'avoiding':
        # Try to maintain visual on target while avoiding
        target_found, target_bbox, target_distance = tracker.find_target(frame, all_detections)
        
        if target_found:
            # Draw target even while avoiding
            cv2.rectangle(frame, 
                         (int(target_bbox[0]), int(target_bbox[1])), 
                         (int(target_bbox[2]), int(target_bbox[3])), 
                         (0, 255, 0), 3)
        
        # Check if obstacle is still blocking
        if obstacle_detected and obstacle_distance < 700:
            # Still blocked - navigate around
            navigate_around_obstacle(obstacle_region, obstacle_distance, depth_colormap)
            cv2.putText(frame, f'AVOIDING - Obstacle {obstacle_distance:.0f}mm {obstacle_region.upper()}', 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            # Path clear - check if target is still visible
            if target_found:
                current_state = 'following'
                print('\nPath clear - Resuming following')
            else:
                # Lost target during avoidance
                robot.stop()
                current_state = 'idle'
                print('\nTarget lost during avoidance')
    
    # Display status
    cv2.putText(frame, f'State: {current_state.upper()}', 
               (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    if obstacle_detected:
        cv2.putText(frame, f'Obstacle: {obstacle_distance:.0f}mm {obstacle_region}', 
                   (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    
    # Display images
    scale = 0.15
    resized_color = cv2.resize(frame, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_color)
    display_depth.value = bgr8_to_jpeg(resized_depth)

camera.observe(main_callback, names=['color_value'])

print('\n=== Part B: Person Following with Obstacle Avoidance ===')
print('Commands:')
print('  l = Lock onto closest person and start following')
print('  s = Stop robot and reset tracker')
print('  r = Reset tracker only')
print('\nFeatures:')
print('  - YOLO for target human tracking with re-identification')
print('  - Depth-based obstacle detection (Tutorial 3 Part B method)')
print('  - Navigates around obstacles while maintaining target lock')
print('  - Smooth differential drive movement')
print('  - Maintains optimal distance (600-900mm)')
print('========================================================')